# CS 521 - HW 1

Code

In [1]:
!pip install tensorboardX numpy torch torchvision matplotlib tqdm wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(28*28, 200)
        self.fc2 = nn.Linear(200,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc(x))
        x = self.fc2(x)
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 51263863.81it/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1781276.29it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 14051406.27it/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3179327.23it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Sequential(
  (0): Normalize()
  (1): Net(
    (fc): Linear(in_features=784, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
  )
)

# Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [9]:
# The last argument 'targeted' can be used to toggle between a targeted and untargeted attack.
def fgsm(model, x, eps):
    #TODO: implement this as an intermediate step of PGD
    model.eval()
    x_adv = x.clone().detach().requires_grad_(True)
    with torch.enable_grad():
        outputs = model(x_adv)
        loss = F.cross_entropy(outputs, y)
        loss.backward()
        grad_sign = x_adv.grad.sign()
        x_adv = x_adv + eps * grad_sign
        x_adv = torch.clamp(x_adv, 0, 1)
    return x_adv.detach()


def pgd_untargeted(model, x, y, *, k=7, eps=0.1, eps_step=0.025):
    model.eval()
    x_adv = x.clone().detach()
    x_orig = x.clone().detach()
    for _ in range(k):
        x_adv.requires_grad_(True)
        outputs = model(x_adv)
        loss = F.cross_entropy(outputs, y)
        model.zero_grad()
        loss.backward()
        grad_sign = x_adv.grad.sign()
        x_adv = x_adv + eps_step * grad_sign
        eta = torch.clamp(x_adv - x_orig, min=-eps, max=eps)
        x_adv = x_orig + eta
        x_adv = torch.clamp(x_adv, 0, 1).detach()
    return x_adv


# Implement Adversarial Training

In [11]:
def train_model(model, num_epochs, enable_defense=True, attack='pgd', eps=0.1):
    # TODO: implement this function that trains a given model on the MNIST dataset.
    # this is a general-purpose function for both standard training and adversarial training.
    # (toggle enable_defense parameter to switch between training schemes)
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            if enable_defense:
                if attack == 'fgsm':
                    data_adv = fgsm(model, data, target, eps=eps)
                elif attack == 'pgd':
                    data_adv = pgd_untargeted(model, data, target, k=7, eps=eps, eps_step=eps/4)
                else:
                    data_adv = data
                output = model(data_adv)
            else:
                output = model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            total_correct += pred.eq(target).sum().item()
        avg_loss = total_loss / len(train_loader)
        accuracy = 100. * total_correct / len(train_dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')



In [10]:
def test_model_on_attacks(model, attack='pgd', eps=0.1):
    # TODO: implement this function to test the robust accuracy of the given model
    # use pgd_untargeted() within this function
    model.eval()
    total_correct = 0
    total_adv_correct = 0
    total = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        total_correct += pred.eq(target).sum().item()
        if attack == 'fgsm':
            data_adv = fgsm(model, data, target, eps=eps)
        elif attack == 'pgd':
            data_adv = pgd_untargeted(model, data, target, k=7, eps=eps, eps_step=eps/4)
        else:
            data_adv = data
        output_adv = model(data_adv)
        pred_adv = output_adv.argmax(dim=1)
        total_adv_correct += pred_adv.eq(target).sum().item()
        total += target.size(0)
    standard_accuracy = 100. * total_correct / total
    adv_accuracy = 100. * total_adv_correct / total
    print(f'Standard Accuracy: {standard_accuracy:.2f}%')
    print(f'Accuracy under attack ({attack}, eps={eps}): {adv_accuracy:.2f}%')



# Study Accuracy, Quality, etc.

Compare the various results and report your observations on the submission.

In [6]:
## train the original model
model = nn.Sequential(Normalize(), Net())
model = model.to(device)
model.train()

train_model(model, 20, False)
torch.save(model.state_dict(), 'weights.pt')

Epoch 1/20, Loss: 0.2393, Accuracy: 92.95%
Epoch 2/20, Loss: 0.1007, Accuracy: 96.87%
Epoch 3/20, Loss: 0.0680, Accuracy: 97.88%
Epoch 4/20, Loss: 0.0512, Accuracy: 98.34%
Epoch 5/20, Loss: 0.0393, Accuracy: 98.69%
Epoch 6/20, Loss: 0.0319, Accuracy: 98.95%
Epoch 7/20, Loss: 0.0258, Accuracy: 99.17%
Epoch 8/20, Loss: 0.0220, Accuracy: 99.28%
Epoch 9/20, Loss: 0.0200, Accuracy: 99.34%
Epoch 10/20, Loss: 0.0158, Accuracy: 99.45%
Epoch 11/20, Loss: 0.0175, Accuracy: 99.42%
Epoch 12/20, Loss: 0.0144, Accuracy: 99.49%
Epoch 13/20, Loss: 0.0124, Accuracy: 99.57%
Epoch 14/20, Loss: 0.0115, Accuracy: 99.60%
Epoch 15/20, Loss: 0.0123, Accuracy: 99.56%
Epoch 16/20, Loss: 0.0094, Accuracy: 99.64%
Epoch 17/20, Loss: 0.0094, Accuracy: 99.69%
Epoch 18/20, Loss: 0.0108, Accuracy: 99.65%
Epoch 19/20, Loss: 0.0107, Accuracy: 99.64%
Epoch 20/20, Loss: 0.0082, Accuracy: 99.74%


In [12]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [13]:
## PGD attack
model = nn.Sequential(Normalize(), Net())
model.load_state_dict(torch.load('weights.pt'))

for eps in [0.05, 0.1, 0.15, 0.2]:
    print(f"\nTesting with epsilon = {eps}")
    test_model_on_attacks(model, attack='pgd', eps=eps)


Testing with epsilon = 0.05
Standard Accuracy: 97.85%
Accuracy under attack (pgd, eps=0.05): 73.20%

Testing with epsilon = 0.1
Standard Accuracy: 97.85%
Accuracy under attack (pgd, eps=0.1): 20.69%

Testing with epsilon = 0.15
Standard Accuracy: 97.85%
Accuracy under attack (pgd, eps=0.15): 2.61%

Testing with epsilon = 0.2
Standard Accuracy: 97.85%
Accuracy under attack (pgd, eps=0.2): 0.18%


In [14]:
## PGD based adversarial training
model = nn.Sequential(Normalize(), Net())
eps = 0.1
train_model(model, 20, True, 'pgd', eps)
torch.save(model.state_dict(), f'weights_AT_{eps}.pt')

Epoch 1/20, Loss: 0.7846, Accuracy: 73.98%
Epoch 2/20, Loss: 0.4934, Accuracy: 83.92%
Epoch 3/20, Loss: 0.4326, Accuracy: 85.94%
Epoch 4/20, Loss: 0.4003, Accuracy: 86.88%
Epoch 5/20, Loss: 0.3808, Accuracy: 87.51%
Epoch 6/20, Loss: 0.3664, Accuracy: 88.04%
Epoch 7/20, Loss: 0.3552, Accuracy: 88.32%
Epoch 8/20, Loss: 0.3464, Accuracy: 88.64%
Epoch 9/20, Loss: 0.3389, Accuracy: 88.96%
Epoch 10/20, Loss: 0.3322, Accuracy: 89.08%
Epoch 11/20, Loss: 0.3260, Accuracy: 89.42%
Epoch 12/20, Loss: 0.3216, Accuracy: 89.33%
Epoch 13/20, Loss: 0.3174, Accuracy: 89.54%
Epoch 14/20, Loss: 0.3130, Accuracy: 89.78%
Epoch 15/20, Loss: 0.3086, Accuracy: 89.85%
Epoch 16/20, Loss: 0.3068, Accuracy: 89.97%
Epoch 17/20, Loss: 0.3029, Accuracy: 89.98%
Epoch 18/20, Loss: 0.3000, Accuracy: 90.13%
Epoch 19/20, Loss: 0.2957, Accuracy: 90.27%
Epoch 20/20, Loss: 0.2953, Accuracy: 90.28%
